In [1]:
import pandas as pd

In [2]:
tweets = pd.read_csv("./utility/data/stockerbot-export1.csv")
tweets

,id,text,timestamp,source,symbols,company_names,url,verified
0,1.0197E+18,VIDEO: “I was in my office. I was minding my o...,Wed Jul 18 21:33:26 +0000 2018,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,TRUE
1,1.01971E+18,The price of lumber $LB_F is down 22% since hi...,Wed Jul 18 22:22:47 +0000 2018,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,TRUE
2,1.01971E+18,Who says the American Dream is dead? https://t...,Wed Jul 18 22:32:01 +0000 2018,TheStreet,AIG,American,https://buff.ly/2L3kmc4,TRUE
3,1.01972E+18,Barry Silbert is extremely optimistic on bitco...,Wed Jul 18 22:52:52 +0000 2018,MarketWatch,BTC,Bitcoin,https://twitter.com/i/web/status/1019716662587...,TRUE
4,1.01972E+18,How satellites avoid attacks and space junk wh...,Wed Jul 18 23:00:01 +0000 2018,Forbes,ORCL,Oracle,http://on.forbes.com/6013DqDDU,TRUE
...,...,...,...,...,...,...,...,...
28435,1.01973E+18,$FB : 29234a9c-7f08-4d5a-985f-cb1a5554ecf9,Wed Jul 18 23:46:13 +0000 2018,test5f1798,FB,Facebook,NaN,FALSE
28436,1.01973E+18,【仮想通貨】ビットコインの価格上昇、８０万円台回復 約１カ月半ぶり $BTC ht...,Wed Jul 18 23:46:19 +0000 2018,keizai_toushi17,BTC,Bitcoin,http://keizai-toushi-navi.com/?p=26838,FALSE
28437,1.01973E+18,RT @invest_in_hd: 'Nuff said! $TEL #telcoin #...,Wed Jul 18 23:46:19 +0000 2018,iad81,BTC,Bitcoin,https://twitter.com/CRYPTOVERLOAD/status/10178...,FALSE
28438,1.01973E+18,【仮想通貨】ビットコインの価格上昇、８０万円台回復 約１カ月半ぶり $BTC ht...,Wed Jul 18 23:46:20 +0000 2018,O8viWMyrCV6cBOZ,BTC,Bitcoin,http://true.velvet.jp/monexx/archives/2357,FALSE


In [3]:
senti = pd.read_csv("./utility/data/tweet_sentiment.csv")
tweets["sentiment"] = senti["sentiment"]

In [4]:
data = tweets
data = data.dropna(axis=0, subset=["text", "sentiment"])
data = data.drop(labels=["company_names", "url", "id"], axis=1)[["timestamp", "source", "verified", "symbols", "sentiment", "text"]]
data = data[data.symbols != 'BTC']
data

,timestamp,source,verified,symbols,sentiment,text
0,Wed Jul 18 21:33:26 +0000 2018,GoldmanSachs,TRUE,GS,0,VIDEO: “I was in my office. I was minding my o...
1,Wed Jul 18 22:22:47 +0000 2018,StockTwits,TRUE,M,0,The price of lumber $LB_F is down 22% since hi...
2,Wed Jul 18 22:32:01 +0000 2018,TheStreet,TRUE,AIG,-1,Who says the American Dream is dead? https://t...
4,Wed Jul 18 23:00:01 +0000 2018,Forbes,TRUE,ORCL,-1,How satellites avoid attacks and space junk wh...
5,Wed Jul 18 23:04:00 +0000 2018,jimcramer,TRUE,FB-GOOGL-GOOG,0,.@RealMoney's David Butler's favorite FANG sto...
...,...,...,...,...,...,...
28427,Wed Jul 18 23:45:53 +0000 2018,WallStJesus,FALSE,JNPR,0,Cisco +3.7% on report Amazon denies plans to c...
28428,Wed Jul 18 23:45:55 +0000 2018,StakNLoot703,FALSE,L,-1,Kill$witch - $L Domo https://t.co/SsFq987in4 v...
28432,Wed Jul 18 23:46:00 +0000 2018,danhyder,FALSE,XOM,1,Exxon Once a ‘Perfect Machine’ Is Running Dry ...
28435,Wed Jul 18 23:46:13 +0000 2018,test5f1798,FALSE,FB,0,$FB : 29234a9c-7f08-4d5a-985f-cb1a5554ecf9


In [13]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def process(tweet):
    # remove handles, urls, and any non-letter characters
    step1 = r'@[A-Za-z0-9]+'
    step2 = r'https?://[A-Za-z0-9./]+'
    step3 = r'[^a-zA-Z]'
    
    re_list = [step1, step2, step3]
    compiled = re.compile('|'.join(re_list))
    
    tweet = tweet.lower()
    stripped = re.sub(compiled, " ", tweet)
    tokens = word_tokenize(stripped)
    
    stop_words = set(stopwords.words("english"))
    
    tokenized_tweets = [token for token in tokens if token not in stop_words]
    
    return tokenized_tweets

    
data["processed_text"] = data["text"].apply(process)
data

,timestamp,source,verified,symbols,sentiment,text,processed_text
0,Wed Jul 18 21:33:26 +0000 2018,GoldmanSachs,TRUE,GS,0,VIDEO: “I was in my office. I was minding my o...,"[video, office, minding, business, david, solo..."
1,Wed Jul 18 22:22:47 +0000 2018,StockTwits,TRUE,M,0,The price of lumber $LB_F is down 22% since hi...,"[price, lumber, lb, f, since, hitting, ytd, hi..."
2,Wed Jul 18 22:32:01 +0000 2018,TheStreet,TRUE,AIG,-1,Who says the American Dream is dead? https://t...,"[says, american, dream, dead]"
4,Wed Jul 18 23:00:01 +0000 2018,Forbes,TRUE,ORCL,-1,How satellites avoid attacks and space junk wh...,"[satellites, avoid, attacks, space, junk, circ..."
5,Wed Jul 18 23:04:00 +0000 2018,jimcramer,TRUE,FB-GOOGL-GOOG,0,.@RealMoney's David Butler's favorite FANG sto...,"[david, butler, favorite, fang, stock, realmon..."
...,...,...,...,...,...,...,...
28427,Wed Jul 18 23:45:53 +0000 2018,WallStJesus,FALSE,JNPR,0,Cisco +3.7% on report Amazon denies plans to c...,"[cisco, report, amazon, denies, plans, compete..."
28428,Wed Jul 18 23:45:55 +0000 2018,StakNLoot703,FALSE,L,-1,Kill$witch - $L Domo https://t.co/SsFq987in4 v...,"[kill, witch, l, domo, via]"
28432,Wed Jul 18 23:46:00 +0000 2018,danhyder,FALSE,XOM,1,Exxon Once a ‘Perfect Machine’ Is Running Dry ...,"[exxon, perfect, machine, running, dry, wall, ..."
28435,Wed Jul 18 23:46:13 +0000 2018,test5f1798,FALSE,FB,0,$FB : 29234a9c-7f08-4d5a-985f-cb1a5554ecf9,"[fb, c, f, f, cb, ecf]"
